In [1]:
from pathlib import Path
import pandas as pd

[CDC Places Data Portal](https://chronicdata.cdc.gov/browse?category=500+Cities+%26+Places)

In [3]:
data_dir = Path(r'../DATA')
cdc_dir = data_dir / 'places'

In [15]:
dfs = []
for fobj in cdc_dir.glob('PLACES*.*'):
    print(f'loading: {fobj.name}')
    year = fobj.name.split('___')[-1].split('_')[0]
    print(f'datayear: {year}')
    df = pd.read_csv(fobj)
    n_rows, n_cols = df.shape
    print(f'loaded dataframe: {n_rows} rows, {n_cols} cols')
    df['year'] = year
    dfs.append(df)
cdc_df = pd.concat(dfs)

loading: PLACES__County_Data__GIS_Friendly_Format___2020_release.csv
datayear: 2020
loaded dataframe: 3142 rows, 118 cols
loading: PLACES__County_Data__GIS_Friendly_Format___2021_release.csv
datayear: 2021
loaded dataframe: 3142 rows, 126 cols


In [16]:
for col in cdc_df.columns:
    print(col)

StateAbbr
StateDesc
CountyName
CountyFIPS
TotalPopulation
ACCESS2_CrudePrev
ACCESS2_Crude95CI
ACCESS2_AdjPrev
ACCESS2_Adj95CI
ARTHRITIS_CrudePrev
ARTHRITIS_Crude95CI
ARTHRITIS_AdjPrev
ARTHRITIS_Adj95CI
BINGE_CrudePrev
BINGE_Crude95CI
BINGE_AdjPrev
BINGE_Adj95CI
BPHIGH_CrudePrev
BPHIGH_Crude95CI
BPHIGH_AdjPrev
BPHIGH_Adj95CI
BPMED_CrudePrev
BPMED_Crude95CI
BPMED_AdjPrev
BPMED_Adj95CI
CANCER_CrudePrev
CANCER_Crude95CI
CANCER_AdjPrev
CANCER_Adj95CI
CASTHMA_CrudePrev
CASTHMA_Crude95CI
CASTHMA_AdjPrev
CASTHMA_Adj95CI
CERVICAL_CrudePrev
CERVICAL_Crude95CI
CERVICAL_AdjPrev
CERVICAL_Adj95CI
CHD_CrudePrev
CHD_Crude95CI
CHD_AdjPrev
CHD_Adj95CI
CHECKUP_CrudePrev
CHECKUP_Crude95CI
CHECKUP_AdjPrev
CHECKUP_Adj95CI
CHOLSCREEN_CrudePrev
CHOLSCREEN_Crude95CI
CHOLSCREEN_AdjPrev
CHOLSCREEN_Adj95CI
COLON_SCREEN_CrudePrev
COLON_SCREEN_Crude95CI
COLON_SCREEN_AdjPrev
COLON_SCREEN_Adj95CI
COPD_CrudePrev
COPD_Crude95CI
COPD_AdjPrev
COPD_Adj95CI
COREM_CrudePrev
COREM_Crude95CI
COREM_AdjPrev
COREM_Adj95CI
COREW_

In [20]:
cdc_df['CountyName'].nunique()

1839

In [21]:
ct_df = cdc_df[cdc_df['StateAbbr'] == 'CT']

In [22]:
ct_df['CountyName'].value_counts()

Fairfield     2
Hartford      2
Litchfield    2
Middlesex     2
New Haven     2
New London    2
Tolland       2
Windham       2
Name: CountyName, dtype: int64

In [23]:
ct_df

,StateAbbr,StateDesc,CountyName,CountyFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ACCESS2_AdjPrev,ACCESS2_Adj95CI,ARTHRITIS_CrudePrev,...,Geolocation,year,DEPRESSION_CrudePrev,DEPRESSION_Crude95CI,DEPRESSION_AdjPrev,DEPRESSION_Adj95CI,GHLTH_CrudePrev,GHLTH_Crude95CI,GHLTH_AdjPrev,GHLTH_Adj95CI
308,CT,Connecticut,Fairfield,9001,943823,10.0,"( 8.5, 11.6)",10.6,"( 9.0, 12.3)",23.4,...,POINT (-73.38942182 41.27166724),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,CT,Connecticut,Hartford,9003,892697,9.8,"( 8.3, 11.5)",10.2,"( 8.6, 12.0)",26.5,...,POINT (-72.73276698 41.80653444),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,CT,Connecticut,Litchfield,9005,181111,7.8,"( 6.3, 9.6)",8.6,"( 7.0, 10.5)",28.2,...,POINT (-73.24529195 41.79234069),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,CT,Connecticut,Middlesex,9007,162682,7.3,"( 6.0, 8.9)",7.9,"( 6.4, 9.6)",28.2,...,POINT (-72.53513114 41.46333347),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,CT,Connecticut,New Haven,9009,857620,10.7,"( 9.0, 12.3)",11.2,"( 9.5, 12.9)",27.3,...,POINT (-72.93263656 41.41052748),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,CT,Connecticut,New London,9011,266784,9.3,"( 7.8, 10.8)",9.8,"( 8.3, 11.5)",28.5,...,POINT (-72.10184166 41.48674009),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,CT,Connecticut,Tolland,9013,150921,7.5,"( 6.1, 9.3)",7.9,"( 6.4, 9.5)",23.6,...,POINT (-72.33645479 41.8545849),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,CT,Connecticut,Windham,9015,117027,9.6,"( 7.9, 11.5)",10.1,"( 8.4, 12.1)",31.2,...,POINT (-71.98753068 41.82994523),2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,CT,Connecticut,Fairfield,9001,943332,13.5,"(11.3, 16.0)",14.2,"(11.8, 16.8)",20.2,...,POINT (-73.38942182 41.27166724),2021,13.4,"(12.7, 14.2)",13.5,"(12.8, 14.3)",14.0,"(12.1, 15.9)",13.3,"(11.6, 15.2)"
749,CT,Connecticut,New Haven,9009,854757,13.3,"(11.4, 15.3)",13.8,"(11.7, 15.9)",23.7,...,POINT (-72.93263656 41.41052748),2021,16.1,"(15.3, 16.9)",16.2,"(15.4, 17.1)",16.7,"(14.8, 18.7)",16.2,"(14.4, 18.1)"
